In [1]:
# test find similarity  img
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
import numpy as np
from numpy import linalg as LA
import os
import h5py
from keras.applications.densenet import DenseNet121,preprocess_input

In [ ]:
# Keras Application model參考 :https://keras.io/api/applications/
class ModelNet:
    def __init__ (self):
        self.input_shape = (224,224,3)
        self.weight = 'imagenet'
        self.pooling = 'max' #池化 'max','avg',None
        # include_top=False，表示會載入模型，不包括加在最後3層的卷積層，通常是取得 Features
        self.model_densenet = DenseNet121(weights = self.weight, 
                                          input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]), 
                                          pooling = self.pooling, 
                                          include_top = False)
#         self.model_densenet.predict(np.zeros((1, 224, 224, 3)))        
    # 提取最後一層conv特徵
    def extract_feat(self, img_path):
        img = keras.preprocessing.image.load_img(img_path,target_size=(self.input_shape[0], self.input_shape[1]))
        img = keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0) #(1, 224, 224, 3)
        img = preprocess_input(img) 
        feat = self.model_densenet.predict(img)
        norm_feat = feat[0]/LA.norm(feat[0])
        return norm_feat


In [ ]:
def preprocess_image(image):
    try:
        # load to gray image
        image = cv2.imread(image)
        ori = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(ori, cv2.COLOR_BGR2GRAY)
        
        # 用Sobel算子計算x，y方向上的梯度，之後在x方向上減去y方向上的梯度
        # 我們留下具有高水平梯度和低垂直梯度的圖像區域
        gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
        # subtract the y-gradient from the x-gradient
        gradient = cv2.subtract(gradX, gradY)
        gradient = cv2.convertScaleAbs(gradient)

        # blur and the image 去除圖像上的噪聲。
        blurred = cv2.blur(gradient, (9, 9))

        # threshold the image 對模糊圖像二值化
        (_, thresh) = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # perform a series of erosions and dilations 
        closed = cv2.erode(closed, None, iterations=4) # 侵蝕
        closed = cv2.dilate(closed, None, iterations=58) # 膨脹

        # find Contour 
        (cnts,_) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        c = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

        # compute the rotated bounding box of the largest contour
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))

        # find image bounding box
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]

        x1 = min(Xs)
        y1 = min(Ys)     
        x2 = max(Xs)
        y2 = max(Ys)

        if x1 < 0:
            x1 = 0
        if y1 < 0:    
            y1 = 0
        
        hight = y2 - y1
        width = x2 - x1
        crop_img = image[y1:y1+hight, x1:x1+width]

#         img_result = tf.image.resize_with_crop_or_pad(crop_img,590,590)
        img_result = tf.image.resize(crop_img,[224,224])
        img_result = np.array(img_result)
        img_result = cv2.cvtColor(img_result,cv2.COLOR_BGR2RGB)
        return img_result
    except IndexError:
        print("list out of range")
        return -1


In [ ]:
def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

In [ ]:
import time
if __name__=="__main__":
    start_time = time.time()
    database = 'images2'
    img_list = get_imlist(database)
    
    feats = []
    names = []
    
    model = ModelNet()
    for i, img_path in enumerate(img_list): 
        try:
#             img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
#             img = keras.preprocessing.image.img_to_array(img)
            norm_feat = model.extract_feat(img_path)      
            img_name = os.path.split(img_path)[1]
            feats.append(norm_feat)
            names.append(img_name)
            print('extracting featurn from image NO. %d , %d images in total' %((i + 1),len(img_list)))
        except:
            pass
#     print(feats)
        
#     feats = np.array(feats)
    model_data = 'models1/feats9.h5' # 存取資料檔名及位置
    
    
    h5f = h5py.File(model_data, 'w')
    h5f.create_dataset('dataset_1',data = feats)
    h5f.create_dataset('dataset_2',data = np.string_(names))

    h5f.close()
    end_time = time.time()
    print((end_time-start_time))

In [ ]:
database = './images3'
img_list = get_imlist(database)
for i, img_path in enumerate(img_list):      
    img_name = os.path.split(img_path)[1]
    print(img_name)

In [ ]:
def cosine_similarity(ratings):
    sim = ratings.dot(ratings.T)
    if not isinstance(sim, np.ndarray):
        sim = sim.toarray()
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:
from sklearn.decomposition import PCA
print(True)

In [ ]:
# 圖片預處理  Model with extract_feat 
def extract_feat1(img_path):
#     input_shape = (224,224,3)    
#     img = keras.preprocessing.image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
#     img = keras.preprocessing.image.img_to_array(img)
    img = preprocess_image(img_path)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    model_densenet = DenseNet121(weights ='imagenet', 
                            input_shape = (224,224,3), 
                            pooling = 'max', include_top = False)
#     self.model_densenet.predict(np.zeros((1, 224, 224, 3)))
    feat = model_densenet.predict(img)
    # print(feat.shape)
    norm_feat = feat[0]/LA.norm(feat[0])
    return norm_feat


In [ ]:
extract_feat1('./data')

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import cv2
# import argparse
start_time = time.time()
query = './database/MDlogo.jpg'
index = 'models1/feats9.h5'
result = 'images2'

h5f = h5py.File(index,'r')
feats = h5f['dataset_1'][:]
# print(feats)
# print(feats[0].shape)
imgNames = h5f['dataset_2'][:]
# print(imgNames)
# h5f.close()


queryDir = args["query"]
queryImg = mpimg.imread(query)
plt.title("Query Image")
plt.imshow(queryImg)
plt.show()


model = ModelNet()

# img_result = preprocess_image(query)
queryVec = model.extract_feat(query)

scores = np.dot(queryVec, feats.T)
rank_ID = np.argsort(scores)[::-1]
rank_score = scores[rank_ID]
# print (rank_ID)
print (rank_score)

maxres = 3          #檢索出三張相似度最高的圖片
imlist = []
for i,index in enumerate(rank_ID[0:maxres]):
    imlist.append(imgNames[index])
#     print(type(imgNames[index]))
    print("top %d "%(i+1)+"image names: "+str(imgNames[index],'utf-8') + " scores: %f"%(rank_score[i]))
#     print("top %d "%(i+1)+" scores: %f"%(rank_score[i]))
# print("top %d images in order are: " %maxres, imlist)

# show top #maxres retrieved result one by one
for i,im in enumerate(imlist):
    image = mpimg.imread(result+"/"+str(im, 'utf-8'))
#     plt.title("search output %d" %(i+1))
    plt.imshow(image)
    plt.show()
end_time = time.time()
print((end_time-start_time))

In [ ]:
# mean pooling
top 1 image names: md2.jpg scores: 0.711408
top 2 image names: md4.jpg scores: 0.687151
top 3 image names: md5.jpg scores: 0.680315
# max pooling
top 1 image names: md2.jpg scores: 0.730196
top 2 image names: md4.jpg scores: 0.720794
top 3 image names: md5.jpg scores: 0.703880

In [ ]:

index = 'models1/feats3.h5'
result = 'images2'
h5f = h5py.File(index,'r')
feats = h5f['dataset_1'][:]
names = h5f['dataset_2'][:]
print(feats.shape)
# for n in
# features_compress = feats.reshape(len(names),)
# sim = cosine_similarity(features_compress)
# sim

In [4]:
# from scipy.sparse import tocsr
query = './database/test12345.jpg'
# print(type(query))
img = tf.keras.preprocessing.image.load_img(query)
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
models = DenseNet121(weights = 'imagenet', 
                      input_shape = (225, 225, 3), 
                      pooling = "max", 
                     include_top = False)
# print(models.summary())
feat = models.predict(img)
norm_feat = feat[0]/LA.norm(feat[0])
norm_feat

array([1.6792641e-05, 1.3465235e-04, 8.1241073e-05, ..., 3.9306588e-02,
       3.0282084e-02, 1.0960010e-02], dtype=float32)

In [10]:
query = './database/3.jpg'
# print(type(query))
img = tf.keras.preprocessing.image.load_img(query)
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
models = DenseNet121(weights = 'imagenet', 
#                       input_shape = (224, 224, 3), 
                      pooling = "max", 
                     include_top = True)
print(models.summary())
feat = models.predict(img)
print('Predicted:', decode_predictions(feat, top=3)[0])
# norm_feat2 = feat[0]/LA.norm(feat[0])
# norm_feat2

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_10 (ZeroPadding2 (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_10[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

conv5_block6_1_conv (Conv2D)    (None, 7, 7, 128)    86016       conv5_block6_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block6_1_bn (BatchNormali (None, 7, 7, 128)    512         conv5_block6_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block6_1_relu (Activation (None, 7, 7, 128)    0           conv5_block6_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block6_2_conv (Conv2D)    (None, 7, 7, 32)     36864       conv5_block6_1_relu[0][0]        
__________________________________________________________________________________________________
conv5_block6_concat (Concatenat (None, 7, 7, 704)    0           conv5_block5_concat[0][0]        
                                                                 conv5_block6_2_conv[0][0]        
__________

NameError: name 'decode_predictions' is not defined

In [27]:
# from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = DenseNet121(weights='imagenet')

img_path ='./database/30.jpg'

img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n06359193', 'web_site', 0.6246015), ('n03710637', 'maillot', 0.12076393), ('n03376595', 'folding_chair', 0.09235375)]


In [31]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = './database/38.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02124075', 'Egyptian_cat', 0.40068024), ('n02123045', 'tabby', 0.30014575), ('n02123159', 'tiger_cat', 0.25841358)]


In [5]:
scores = np.dot(norm_feat,norm_feat2)
scores

0.5408155

In [ ]:
from PIL import Image
import numpy as np
#開啟一張圖片
img = Image.open("./database/MDlogo.jpg")
#將圖片化為32*21的
img = img.resize((32, 32))
#二值化
img = img.point(lambda x:1 if x > 120 else 0) 
#將圖片轉換為陣列形式，元素為其畫素的亮度值
img_array = np.asarray(img)
print(img_array)

In [ ]:
import cv2
image = './database/MDlogo.jpg'
image = cv2.imread(image)
ori = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(ori, cv2.COLOR_BGR2GRAY)

# 用Sobel算子計算x，y方向上的梯度，之後在x方向上減去y方向上的梯度
# 我們留下具有高水平梯度和低垂直梯度的圖像區域
gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
# subtract the y-gradient from the x-gradient
gradient = cv2.subtract(gradX, gradY)
gradient = cv2.convertScaleAbs(gradient)

# blur and the image 去除圖像上的噪聲。
blurred = cv2.blur(gradient, (9, 9))

# threshold the image 對模糊圖像二值化
(_, thresh) = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# perform a series of erosions and dilations 
closed = cv2.erode(closed, None, iterations=4) # 侵蝕
closed = cv2.dilate(closed, None, iterations=58) # 膨脹

# find Contour 
(cnts,_) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
c = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

# find image bounding box
Xs = [i[0] for i in box]
Ys = [i[1] for i in box]

x1 = min(Xs)
y1 = min(Ys)     
x2 = max(Xs)
y2 = max(Ys)

if x1 < 0:
    x1 = 0
if y1 < 0:    
    y1 = 0

hight = y2 - y1
width = x2 - x1
crop_img = image[y1:y1+hight, x1:x1+width]
# crop_img
#         img_result = tf.image.resize_with_crop_or_pad(crop_img,590,590)
img_result = tf.image.resize(crop_img,[224,224])
img_result = np.array(img_result)
img_result = cv2.cvtColor(img_result,cv2.COLOR_BGR2RGB)
print(img_result)

test

In [ ]:
# Xception
12.962688446044922
2.912721633911133
image names: b'089037480.jpg' scores: 0.659446
image names: b'089037550a.jpg' scores: 0.623441
image names: b'089037550.jpg' scores: 0.575833

In [ ]:
#vgg16 
16.262974500656128
1.196272373199463
image names: b'Md1.jpg' scores: 0.560349
image names: b'089037479.jpg' scores: 0.543888
image names: b'md3.jpg' scores: 0.516564

In [ ]:
# VGG19
19.609010934829712
1.4062960147857666
image names: b'Md1.jpg' scores: 0.648825
image names: b'089037525.jpg' scores: 0.539406
image names: b'089037526.jpg' scores: 0.520872

In [ ]:
#resnet50
12.594780445098877
3.3022353649139404
image names: b'Md1.jpg' scores: 0.631384
image names: b'md4.jpg' scores: 0.613644
image names: b'md5.jpg' scores: 0.595053

In [ ]:
# ResNet101
20.10851001739502
5.994015216827393
image names: b'089037473.jpg' scores: 0.623724
image names: b'md3.jpg' scores: 0.600736
image names: b'md4.jpg' scores: 0.599365

In [ ]:
# ResNet152
27.159053325653076
8.69070816040039
image names: b'Md1.jpg' scores: 0.634179
image names: b'089037511.jpg' scores: 0.619060
image names: b'089037508.jpg' scores: 0.618733

In [ ]:
# ResNet50V2
12.57044243812561
3.2562265396118164
image names: b'089037509.jpg' scores: 0.627007
image names: b'089037508.jpg' scores: 0.607932
image names: b'089037510.jpg' scores: 0.599109

In [ ]:
# ResNet101V2
19.84201669692993
5.342225074768066
image names: b'089037509.jpg' scores: 0.589442
image names: b'089037508.jpg' scores: 0.543976
image names: b'089037511.jpg' scores: 0.531375

In [ ]:
# ResNet152V2
26.78176522254944
7.997605800628662
image names: b'089037480.jpg' scores: 0.636117
image names: b'md4.jpg' scores: 0.596959
image names: b'md3.jpg' scores: 0.593900

In [ ]:
# InceptionV3
11.80576753616333
5.238420486450195
image names: b'089037509.jpg' scores: 0.740130
image names: b'089037527.jpg' scores: 0.736606
image names: b'089037479.jpg' scores: 0.734550


In [ ]:
#inception_resnet_v2
22.869795560836792
9.946449041366577
image names: b'089037479.jpg' scores: 0.770759
image names: b'089037511.jpg' scores: 0.758921
image names: b'089037509.jpg' scores: 0.757221

In [ ]:
# modilenet
7.191591024398804
1.6735904216766357
image names: b'Md1.jpg' scores: 0.688429
image names: b'089037526.jpg' scores: 0.687456
image names: b'089037529.jpg' scores: 0.682550

In [ ]:
# mobilenet_v2 
8.237006187438965
2.547529935836792
image names: b'089037550a.jpg' scores: 0.715992
image names: b'089037550.jpg' scores: 0.706605
image names: b'089037527.jpg' scores: 0.691365

In [ ]:
# DenseNet121 (200,200)
15.37054967880249
5.748606443405151
(250,250)
image names: b'md2.jpg' scores: 0.716354
image names: b'Md1.jpg' scores: 0.716196
image names: b'md5.jpg' scores: 0.697024
(240,240)
image names: b'md5.jpg' scores: 0.732438
image names: b'md4.jpg' scores: 0.729016
image names: b'md2.jpg' scores: 0.704233
(230,230)
image names: b'md4.jpg' scores: 0.723096
image names: b'md2.jpg' scores: 0.709047
image names: b'md3.jpg' scores: 0.704193
(224,224)
image names: b'md2.jpg' scores: 0.730196
image names: b'md4.jpg' scores: 0.720794
image names: b'md5.jpg' scores: 0.703880
(200,200)
image names: b'md3.jpg' scores: 0.713901
image names: b'md5.jpg' scores: 0.703691
image names: b'md4.jpg' scores: 0.698603
(180,180)
image names: b'Md1.jpg' scores: 0.679671
image names: b'md3.jpg' scores: 0.622696
image names: b'089037480.jpg' scores: 0.613786
(150,150)
image names: b'Md1.jpg' scores: 0.679671
image names: b'md3.jpg' scores: 0.622696
image names: b'089037480.jpg' scores: 0.613786

In [ ]:
# DenseNet169
20.443771362304688
8.133271217346191
image names: b'089037473.jpg' scores: 0.745333
image names: b'089037474.jpg' scores: 0.733137
image names: b'089037472.jpg' scores: 0.719623

In [ ]:
# DenseNet201
22.786865234375
9.54400086402893
image names: b'089037475.jpg' scores: 0.663582
image names: b'089037474.jpg' scores: 0.655224
image names: b'089037472.jpg' scores: 0.631602

In [ ]:
# NASNetMobile
16.750481367111206
9.552340269088745
image names: b'089037548.jpg' scores: 0.773100
image names: b'089037473.jpg' scores: 0.760417
image names: b'089037547.jpg' scores: 0.758165

In [ ]:
# NASNetLarge
ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (331, 331, 3).

In [ ]:
#DenseNet121
image names: b'md2.jpg' scores: 0.730196
image names: b'md4.jpg' scores: 0.720794
image names: b'md5.jpg' scores: 0.703880
#resnet50
image names: b'Md1.jpg' scores: 0.631384
image names: b'md4.jpg' scores: 0.613644
image names: b'md5.jpg' scores: 0.595053
# modilenet
image names: b'Md1.jpg' scores: 0.688429
image names: b'089037526.jpg' scores: 0.687456
image names: b'089037529.jpg' scores: 0.682550       
# ResNet152V2
image names: b'089037480.jpg' scores: 0.636117
image names: b'md4.jpg' scores: 0.596959
image names: b'md3.jpg' scores: 0.593900
# ResNet152
image names: b'Md1.jpg' scores: 0.634179
image names: b'089037511.jpg' scores: 0.619060
image names: b'089037508.jpg' scores: 0.618733
# ResNet101
image names: b'089037473.jpg' scores: 0.623724
image names: b'md3.jpg' scores: 0.600736
image names: b'md4.jpg' scores: 0.599365
# VGG19
image names: b'Md1.jpg' scores: 0.648825
image names: b'089037525.jpg' scores: 0.539406
image names: b'089037526.jpg' scores: 0.520872
# vgg16 
image names: b'Md1.jpg' scores: 0.560349
image names: b'089037479.jpg' scores: 0.543888
image names: b'md3.jpg' scores: 0.516564